In [61]:
from mobile import get_brands
import os
import glob
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import unicodedata
from pprint import pprint

In [3]:
all_brands = get_brands()

In [4]:
all_brands.head()

,url,brand,n_models
0,https://www.gsmarena.com/acer-phones-59.php,Acer,100
1,https://www.gsmarena.com/alcatel-phones-5.php,alcatel,378
2,https://www.gsmarena.com/allview-phones-88.php,Allview,134
3,https://www.gsmarena.com/amazon-phones-76.php,Amazon,17
4,https://www.gsmarena.com/amoi-phones-28.php,Amoi,47


In [5]:
expected_n_models = all_brands.n_models.sum()
n_brands = all_brands.shape[0]
print (expected_n_models, n_brands)

9505 110


In [6]:
if os.path.isdir('model_attributes'):
    print ('Results exist!')

Results exist!


In [7]:
all_results = glob.glob('model_attributes/*.json')
 # number of results matches brands
assert len(all_results) == n_brands, "Wrong number of brands"

# How subsequent read operation works

In [64]:
test_fn = 'model_attributes/Acer_models.json'
io = open(test_fn, 'r')
to_json = [json.loads(line) for line in io] # new line per record
test_df = json_normalize(to_json)
# check acer brands
(test_df.shape, all_brands.loc[all_brands.brand == 'Acer'].n_models.values[0]) # matching n.models

((100, 195), 100)

# Combining all data

In [9]:
from functools import reduce

def read_fp(fn): return [json.loads(line) for line in open(fn, 'r') if line]

all_records = list(map(read_fp, all_results))

# Mistakenly hardcoded brand name, stretching to work with zip, making amendments on fly
brand_names = [x.split('/')[-1].replace('_models.json','') for x in all_results]
stretched_brands = [[b] * len(objs) for b,objs in zip(brand_names, all_records)]
stretched_brands = [x for y in stretched_brands for x in y]

all_records = reduce(lambda x,y: x+y, all_records) # combining all lists
all_records = [{**r, **{'brand' : brand}} for r,brand in zip(all_records, stretched_brands)]

In [10]:
normalized = json_normalize(all_records) # flattens dictionaries into df

# All results in one DF

In [11]:
normalized.head()

,Battery.Music play.attr_meta,Battery.Music play.attr_value,Battery.Music play.sub_cat_meta,Battery.Stand-by.attr_meta,Battery.Stand-by.attr_value,Battery.Stand-by.sub_cat_meta,Battery.Talk time.attr_meta,Battery.Talk time.attr_value,Battery.Talk time.sub_cat_meta,Battery. .attr_meta,...,Tests.null.sub_cat_meta,brand,date,err,fail,hits,model_name,model_page,model_url,trend
0,NaN,NaN,NaN,NaN,NaN,NaN,battalktime1,Up to 9 h (multimedia),talk-time,batdescription1,...,NaN,Acer,NaN,NaN,False,"71,017 hits",Chromebook Tab 10,https://www.gsmarena.com/acer_chromebook_tab_1...,NaN,0.5%
1,NaN,NaN,NaN,NaN,NaN,NaN,battalktime1,Up to 9 h (multimedia),talk-time,batdescription1,...,NaN,Acer,NaN,NaN,False,"341,649 hits",Iconia Talk S,https://www.gsmarena.com/acer_iconia_talk_s-83...,NaN,0.5%
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,batdescription1,...,NaN,Acer,NaN,NaN,False,"508,813 hits",Liquid Z6 Plus,https://www.gsmarena.com/acer_liquid_z6_plus-8...,NaN,0.5%
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,batdescription1,...,NaN,Acer,NaN,NaN,False,"179,222 hits",Liquid Z6,https://www.gsmarena.com/acer_liquid_z6-8304.php,NaN,0.2%
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,batdescription1,...,NaN,Acer,NaN,NaN,False,"245,233 hits",Iconia Tab 10 A3-A40,https://www.gsmarena.com/acer_iconia_tab_10_a3...,NaN,0.2%


# Characteristics

In [78]:
use_columns_count = [c for c in normalized.columns if c.endswith('meta') is True] 
# meta is just another name for subcategory so we exclude it

normalized.count().drop(labels=use_columns_count).sort_values(ascending = False).iloc[:20] 

brand                            9287
model_page                       9287
model_name                       9287
fail                             9287
Launch.Status.attr_value         9279
Display.Size.attr_value          9279
Display.Type.attr_value          9279
Features.Browser.attr_value      9279
Features.Messaging.attr_value    9279
Launch.Announced.attr_value      9279
trend                            9279
Memory.Card slot.attr_value      9279
Network.GPRS.attr_value          9279
Misc.Colors.attr_value           9279
Network.2G bands.attr_value      9279
Network.Technology.attr_value    9279
Sound.Alert types.attr_value     9279
Sound.Loudspeaker .attr_value    9279
Display.Resolution.attr_value    9279
Comms.WLAN.attr_value            9279
dtype: int64

# Major and sub-categories

### Note the empty strings, maybe taking meta attribute here makes sense

In [82]:
all_major_categories = [c.split('.')[0] for c in normalized.columns if c.count('.') > 0]
all_major_categories = set(all_major_categories)
sub_categories = {}
for c in all_major_categories:
    subfields = normalized.head().filter(regex=r'%s.' % c).columns.tolist() 
    subfields = [re.split(r'\.', sub)[1] for sub in subfields if sub.endswith('value')]
    subfields = [unicodedata.normalize('NFKD', c) for c in subfields] # double space needs decoding 
    sub_categories[c] = subfields
    # some without subcategories, i.e. empty string or double empty decoded

In [81]:
pprint (sub_categories)

{'Battery': ['Music play', 'Stand-by', 'Talk time', ' ', 'Battery life'],
 'Body': ['Build', 'Dimensions', 'Keyboard', 'SIM', 'Weight', ' '],
 'Camera': [' ',
            '',
            'Dual',
            'Features',
            'Four',
            'Single',
            'Triple',
            'Video',
            ' ',
            'Camera'],
 'Comms': ['Bluetooth', 'GPS', 'Infrared port', 'NFC', 'Radio', 'USB', 'WLAN'],
 'Display': ['Multitouch',
             'Protection',
             'Resolution',
             'Size',
             'Type',
             ' ',
             'Display'],
 'Features': ['Alarm',
              'Browser',
              'Clock',
              'Games',
              'Java',
              'Languages',
              'Messaging',
              'Sensors',
              ' ',
              'Features',
              'Features'],
 'Launch': ['Announced', 'Status'],
 'Main Camera': ['',
                 'Dual',
                 'Features',
                 'Four',
       

# Getting columns by subcategory

In [83]:
normalized.filter(regex='Battery.Stand-by').sample(n=100)

,Battery.Stand-by.attr_meta,Battery.Stand-by.attr_value,Battery.Stand-by.sub_cat_meta
3336,batstandby1,Up to 410 h (2G) / Up to 410 h (3G),stand-by-time
8589,NaN,NaN,NaN
4221,batstandby1,Up to 200 h,stand-by-time
3878,batstandby1,Up to 150 h,stand-by-time
4321,NaN,NaN,NaN
7214,batstandby1,Up to 250 h,stand-by-time
7760,batstandby1,Up to 1500 h,stand-by-time
1687,batstandby1,Up to 500 h,stand-by-time
5468,NaN,NaN,NaN
3808,batstandby1,Up to 220 h,stand-by-time


# Looking for request failures

In [67]:
normalized.loc[normalized.err.notnull()].groupby(['err','brand']).size() # failed requests

err                                                  brand  
<urlopen error [Errno 54] Connection reset by peer>  O          1
                                                     OnePlus    1
                                                     Oppo       2
                                                     Orange     2
                                                     Samsung    1
Remote end closed connection without response        Oppo       1
dtype: int64

In [68]:
results_df = normalized.groupby('brand').size().rename('results_n').reset_index()
final_df = pd.merge(results_df, all_brands, on='brand')[['brand', 'url', 'results_n', 'n_models']]

In [71]:
final_df.head(100)

,brand,url,results_n,n_models
0,AT&T;,https://www.gsmarena.com/at&t-phones-57.php,4,4
1,Acer,https://www.gsmarena.com/acer-phones-59.php,100,100
2,Allview,https://www.gsmarena.com/allview-phones-88.php,134,134
3,Amazon,https://www.gsmarena.com/amazon-phones-76.php,17,17
4,Amoi,https://www.gsmarena.com/amoi-phones-28.php,47,47
5,Apple,https://www.gsmarena.com/apple-phones-48.php,63,63
6,Archos,https://www.gsmarena.com/archos-phones-90.php,39,39
7,Asus,https://www.gsmarena.com/asus-phones-46.php,169,169
8,BLU,https://www.gsmarena.com/blu-phones-67.php,279,279
9,BQ,https://www.gsmarena.com/bq-phones-108.php,20,20


In [75]:
final_df.head()
print (final_df.shape) # all brands accounted for

(110, 4)


# Missing models

In [76]:
final_df.columns

Index(['brand', 'url', 'results_n', 'n_models'], dtype='object')

In [77]:
# fail safe for additional models added, must be something wrong with pages
mask = final_df['results_n'] != final_df['n_models']
final_df.where(mask).dropna(axis=0, how='all')

,brand,url,results_n,n_models
35,Huawei,https://www.gsmarena.com/huawei-phones-58.php,288.0,289.0
45,Lenovo,https://www.gsmarena.com/lenovo-phones-73.php,180.0,181.0
55,Motorola,https://www.gsmarena.com/motorola-phones-4.php,459.0,461.0
59,Nokia,https://www.gsmarena.com/nokia-phones-1.php,465.0,467.0
61,O,https://www.gsmarena.com/o2-phones-30.php,45.0,245.0
62,OnePlus,https://www.gsmarena.com/oneplus-phones-95.php,8.0,9.0
65,Palm,https://www.gsmarena.com/palm-phones-27.php,16.0,17.0
77,Samsung,https://www.gsmarena.com/samsung-phones-9.php,1155.0,1157.0
83,Sony,https://www.gsmarena.com/sony-phones-7.php,136.0,137.0
98,XOLO,https://www.gsmarena.com/xolo-phones-85.php,80.0,81.0


# Completed

In [79]:
final_df.where(~mask).dropna(axis=0, how = 'all')

,brand,url,results_n,n_models
0,AT&T;,https://www.gsmarena.com/at&t-phones-57.php,4.0,4.0
1,Acer,https://www.gsmarena.com/acer-phones-59.php,100.0,100.0
2,Allview,https://www.gsmarena.com/allview-phones-88.php,134.0,134.0
3,Amazon,https://www.gsmarena.com/amazon-phones-76.php,17.0,17.0
4,Amoi,https://www.gsmarena.com/amoi-phones-28.php,47.0,47.0
5,Apple,https://www.gsmarena.com/apple-phones-48.php,63.0,63.0
6,Archos,https://www.gsmarena.com/archos-phones-90.php,39.0,39.0
7,Asus,https://www.gsmarena.com/asus-phones-46.php,169.0,169.0
8,BLU,https://www.gsmarena.com/blu-phones-67.php,279.0,279.0
9,BQ,https://www.gsmarena.com/bq-phones-108.php,20.0,20.0
